In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os

drivers= {'Driver Number':['44','77','16','5','33','23','55','4','3','31','10','26','11','18','7','99','20','8','63','6'],
          'Driver':['Lewis Hamilton', 'Valtteri Bottas','Charles Leclerc','Sebastian Vettel','Max Verstappen','Alexander Albon','Carlos Sainz','Lando Norris','Daniel Ricciardo','Esteban Ocon','Pierre Gasly','Daniil Kvyat','Sergio Perez','Lance Stroll','Kimi Raikonnen','Antonio Giovinazzi','Kevin Magnussen','Romain Grosjean','George Russell','Nicholas Latifi'],
          'Team':['Mercedes', 'Mercedes', 'Ferrari','Ferrari','Red Bull','Red Bull','Mclaren','Mclaren','Renault','Renault','Alpha Tauri','Alpha Tauri','Racing Point','Racing Point','Alfa Romeo','Alfa Romeo','Haas','Haas','Williams','Williams'],
          'Engine':['Mercedes', 'Mercedes', 'Ferrari','Ferrari','Honda','Honda','Renault','Renault','Renault','Renault','Honda','Honda','Mercedes', 'Mercedes', 'Ferrari','Ferrari','Ferrari','Ferrari','Mercedes', 'Mercedes']     
}

Driver=pd.DataFrame(drivers)

def get_race(url,country=None,session=None):
    response=requests.get(url)
    soup=BeautifulSoup(response.text,'lxml')
    for table in soup.find_all('table'):
        n_columns = 0
        n_rows=0
        column_names = []
        df=[]
        for row in table.find_all('tr'):
            th_tags = row.find_all('th')
        #Determine the number of rows in the table
            td_tags = row.find_all('td')
            rows=[i.text for i in td_tags if i.text!='']
            if rows!=[]:
                df.append(rows)
            df2=df[0:]
            if len(td_tags) > 0:
                n_rows+=1
                if n_columns == 0:
                # Set the number of columns for our table
                    n_columns = len(td_tags)-2    
                    # Handle column names if we find them

            if len(th_tags) > 0 and len(column_names) == 0:
                for th in th_tags:
                    if th.get_text()!='':
                        column_names.append(th.get_text())
                    else:
                        None


    dt=pd.DataFrame(df2,columns = column_names,index= range(0,n_rows))
    dt['Driver'] = dt['Driver'].map(lambda x: x.lstrip('\n').rstrip('\n'))
    
    #The commented line below is for race results
    if session=='race':
        fp=pd.merge(Driver,dt[['No','Pos','Laps','Time/Retired','PTS']],left_on='Driver Number', right_on='No',how='left')
        final=fp[['Driver Number','Driver','Team','Engine','Pos','Laps','Time/Retired','PTS']]
    elif session=='FP1' or session=='FP2' or session=='FP3':
        fp=pd.merge(Driver,dt[['No','Pos','Laps','Time','Gap']],left_on='Driver Number', right_on='No',how='left')
        final=fp[['Driver Number','Driver','Team','Engine','Pos','Laps','Time','Gap']]
    else:
        fp=pd.merge(Driver,dt[['No','Pos','Q1','Q2','Q3','Laps']],left_on='Driver Number', right_on='No',how='left')
        final=fp[['Driver Number','Driver','Team','Engine','Pos','Q1','Q2','Q3','Laps']]
    
    #Note to Self: Edit this to reflect Race Name
    file_path='C:\\Users\\micha\\Dropbox\\F1 Betting and Stats'
    excel_filename=os.path.join(file_path, country+'_'+session+'.xlsx')
    export_excel=final.to_excel(excel_filename, index = None, header=True)
    return(final)

    
    
    

In [6]:
get_race('https://www.formula1.com/en/results.html/2019/races/1020/abu-dhabi/race-result.html','ABU','race')

,Driver Number,Driver,Team,Engine,Pos,Laps,Time/Retired,PTS
0,44,Lewis Hamilton,Mercedes,Mercedes,1,55,1:34:05.715,26
1,77,Valtteri Bottas,Mercedes,Mercedes,4,55,+44.379s,12
2,5,Sebastian Vettel,Ferrari,Ferrari,5,55,+64.357s,10
3,16,Charles Leclerc,Ferrari,Ferrari,3,55,+43.435s,15
4,33,Max Verstappen,Red Bull,Honda,2,55,+16.772s,18
5,23,Alexander Albon,Red Bull,Honda,6,55,+69.205s,8
6,3,Daniel Ricciardo,Renault,Renault,11,54,+1 lap,0
7,27,Nico Hulkenberg,Renault,Renault,12,54,+1 lap,0
8,8,Romain Grosjean,Haas,Ferrari,15,54,+1 lap,0
9,20,Kevin Magnussen,Haas,Ferrari,14,54,+1 lap,0
